In [11]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# CRUD Python import 
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connect to database via CRUD Module

userName = "aacuser"
password = "COFFEE12"
userHost = "nv-desktop-services.apporto.com"
userPort = 30144
userDb = "AAC"
userCol = "animals"
animal = AnimalShelter(userName, password, userHost, userPort, userDb, userCol)




# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(animal.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo Edit.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Center(html.B(html.H2('Developed by James Jones'))),
    html.Hr(),
    
    #Drop Down Menu to filter by rescue type
    html.Div(dcc.Dropdown([{'label':'Water', 'value': 'water'},
                           {'label': 'Mountain/Wilderness', 'value': 'mountain'},
                           {'label':'Disaster/Individual Tracking', 'value': 'tracking'}],
                          placeholder='Rescue Type', id="filter_type",
                         style={'width':'35%'})

    ),
    html.Hr(),
    #Code for the interactive datatale
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         style_table={'overflowX': 'auto'},
                         style_data={'color': 'black', 'backgroundColor': 'white'},
                         style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(220, 220, 220)'}],
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=5

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id','data'),
    [Input('filter_type', 'value')])
def update_dashboard(value):
## FIX ME Add code to filter interactive data table with MongoDB queries
        df = pd.DataFrame.from_records(animal.read({}))
        
        #filter for water rescue animal candidates
        waterFilter = {'$or':
                      [{"breed": "Labrador Retriever Mix"},
                       {"breed":"Chesapeake Bay Retriever"},
                       {"breed": "Newfoundland"}],
                       "sex_upon_outcome": "Intact Female",
                       "age_upon_outcome_in_weeks":{"$gte": 26, "$lte": 156}}
        #filter for mountain and wilderness rescue animal candidates
        mountainFilter = {'$or':
                      [{"breed": "German Shephard"},
                       {"breed": "Alaskan Malamute"},
                       {"breed": "Siberian Husky"},
                       {"breed": "Rottweiler"},
                       {"breed": "Old Endlish Sheepdog"}],
                       "sex_upon_outcome": "Intact Male",
                       "age_upon_outcome_in_weeks":{"$gte": 26, "$lte": 156}}
        #filter for tracking animal candidates
        trackingFilter = {'$or':
                      [{"breed": "German Shephard"},
                       {"breed": "Doberman Pinscher"},
                       {"breed": "Golden Retriever"},
                       {"breed": "Bloodhound"},
                       {"breed": "Rottweiler"}],
                       "sex_upon_outcome": "Intact Male",
                       "age_upon_outcome_in_weeks":{"$gte": 20, "$lte": 300}}
        #if statements to filter results
        if(value == "water"):
            df = pd.DataFrame.from_records(animal.read(waterFilter))
        elif(value == "mountain"):
            df= pd.DataFrame.from_records(animal.read(mountainFilter))
        elif(value == "tracking"):
            df= pd.DataFrame.from_records(animal.read(trackingFilter))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
            
        
        df.drop(columns=['_id'], inplace=True)
        return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):

    #code for basic pie chart
    df = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 14 and 15 define the grid-coordinates for 
            # the map
            # Column 5 defines the breed for the animal
            # Column 10 defines the name of the animal
            # Depending on how the database information is passed this function
            # type conversions may be necessary 
            dl.Marker(position=[float(dff.iloc[row,13]),float(dff.iloc[row,14])],
               children=[
               dl.Tooltip(str(dff.iloc[row,4])),
               dl.Popup([
                  html.H1("Animal Name"),
                  html.P(str(dff.iloc[row,9]))
         ])
      ])
   ])
]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:11014/
